In [51]:
#######################################################################
# This script was written by Nana Owusu, it is meant to visualize     #
# waveform information from XML files produced by the GE scanner's    #
# plotter tool.                                                       #
#######################################################################
# Modules for text interpretation and math
import os, sys, re, fnmatch
import numpy as np
# Modules for plotting and reading xmls
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib import style
import xml.etree.ElementTree as ET
%matplotlib inline
from ipywidgets import interactive
# Module for GUI
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
# Modules for interactive plotting in GUI
import matplotlib.backends.backend_tkagg
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg as FigCanvas
from matplotlib.backends.backend_tkagg import NavigationToolbar2Tk as NavTb2

## Load XML files

In [3]:
LARGE_FONT = ("Verdana", 12)
style.use("ggplot")

# Function for picking XML file order
def xmlItemize(convention,text):
    # Regular expressions for naming convention of shots
    # recorded by the scanner
    if convention == 0:
        pattern = '\w+\.xml\.(\d+)'
        # Identify the time point of each shot/time-point recorded
        token = re.search(pattern, text)
    elif convention == 1:
        pattern = '\w+\.xml'
        # Identify the time point of each shot/time-point recorded
        token = re.search(pattern, text)
    elif convention == 2:
        noExt = 1

    
    if token:
        return int(token.group(1))
    elif noExt:
        return text
    else:
         raise UserWarning('The naming convention must match the regular expression {0}'.format(pattern))

def xmlSort(order,fileList):
    idxNList = zip(order, fileList)
    sortedList = [x for x in idxNList]
    sortedList.sort(key=lambda sortedList:sortedList[0])
    return list(sortedList)

In [4]:
# xmlList = []
# xmlList.append(sourceFolder)

In [5]:
# # Create placeholder and fill list for xmls
# tempList = []
# xmlFiles = []

# # provide the absolute path of the current directory
# imLoc = xmlList[0] + '/'
# # list all files in the XML directory
# dirList = os.listdir(imLoc)
# # store only XML files
# style = int()
# if len(fnmatch.filter(dirList,'*.xml*')) > 1:
#     filesInDir = fnmatch.filter(dirList,'*.xml.*[^0-9]')
#     style = 0
# elif len(fnmatch.filter(dirList,'*.xml*')) == 1:
#     filesInDir = fnmatch.filter(dirList,'*.xml*')
#     style = 1
# elif len(fnmatch.filter(dirList,'*')) > 0:
#     filesInDir = []
#     files = fnmatch.filter(dirList,'*')
#     for x in files:
#         if (ET.parse(x)):
#             filesInDir.append(x)
#         else:
#             continue
# else :
#     raise UserWarning('Found no XML files or the directory was empty.\n')

# for j,k in enumerate(filesInDir):
#     tempList.append(xmlItemize(style,k))
#     xmlFiles.append(k)

# sortedFiles = xmlSort(tempList, xmlFiles)

# # Array for the sorted abs. path for each XML image
# xmlFullPath = []

# # Fill array with XML abs. paths
# x = 0
# stopCond = len(sortedFiles)
# for i,j in sortedFiles:
#     xmlFullPath.append(xmlList[0] + '/' + j)

    
# # Garbage collection of unused variables
# # del tempList, xmlList, sortedFiles

## Load files

In [6]:
# load an XML for info on first plotter time-point
def xmlRoot(xmlSets, numOfShots):
    ''' Takes: 
            list of XML global addresses (xmlSets),
            file count representing the shots
            acquired (numOfShots).
        Returns: 
            sequencer information for each shot (root).'''
    
    # Sequencer information can be learnt from the 
    # code below.
    # for child in root:
    #    print(child.tag, child.attrib)
    # "root" here is from having the ElementTree instance
    # call the getroot() function.

    # Placeholders for XML section headers (tree) and
    # the Sequencer section header (root) info
    tree = []
    root = [[] for x in range(numOfShots)]
    
    # Fill placeholder with ElementTree objects
    # which represent section headers
    for x,y in enumerate(xmlSets):
        tree.append(ET.parse(y))
    
    # Fill placeholder with data for each Sequencer 
    for i,j in enumerate(tree):
        root[i] = j.getroot()
    
    del tree
    return root

# wvm = xmlRoot(xmlFullPath,stopCond)

# seqr = {0: 'SSP', 1: 'XGRAD', 2: 'YGRAD', 3: 'ZGRAD',\
#         4: 'RHO1', 5: 'RHO2', 6: 'THETA1', 7: 'THETA2'}

In [63]:
def extractWvfm(waveObjs, seq, numOfShots):
    ''' Takes: 
            list of sequencer ElementTree objects for
            each time point (waveObjs),
            Sequencer name (seq),
            and XML file count (numOfShots).
        Returns: 
            Numpy array with abscissa and range of a
            particular Sequencer for all time points.'''
    
    # Placeholder of the data points for each Sequencer.
    oneDimStore = [[] for x in range(numOfShots)]
    twoDimStore = []
    
    # Use splitlines utility to interperate whitespace
    # notation in waveform values for a particular Sequencer.
    # Fill nested list with time/amplitude data for each time point.
    t = 0
    while t != numOfShots:
        for x in waveObjs[t][seq][0].itertext():
            oneDimStore[t].append(x.splitlines())
        t += 1

    # Placeholder for time and amplitude of the Sequencer at
    # each time point.
    
    shotLen = 0
    t = 0
    while t != numOfShots:
        # Find the max length of the lists
        for i in iter(oneDimStore[t][0]):
            twoDimStore.append(i.split(' '))
            
        nextShotLen = len(twoDimStore)

        if nextShotLen > shotLen:
            shotLen = nextShotLen

        twoDimStore.clear()
        t += 1
        
    waveToPlot = np.zeros((numOfShots,2,shotLen-1))
    
    # Use split utility to place time (abscissa) and amplitude
    # (range) values into separate rows for each time point.
    t = 0
    while t != numOfShots:
        for x,y in enumerate(oneDimStore[t][0]):
            twoDimStore.append(y.split(' '))
        for x in range(1,len(twoDimStore)):
            waveToPlot[t][0][x-1] = float(twoDimStore[x][0])
            waveToPlot[t][1][x-1] = float(twoDimStore[x][1])
        twoDimStore.clear()
        t += 1
           
    del oneDimStore, twoDimStore
    return waveToPlot

## Plot extracted sequencer waveform

### Procedures for comparing waveforms

In [64]:
## find from the end of an array the contiguous time-point that is < 799.0
def sspStopTime(waveObjs,numOfShots):
    ''' Takes:
            list of sequencer ElementTree objects for
            each time point (waveObjs),
            XML file count (numOfShots)
        Returns:
            list of first time points 
            > 799.0 for each shot (lTime)'''
    
    # Waveform of the SSP board
    sspWv = extractWvfm(waveObjs, 0, numOfShots)
    
    # Storage for true final time points
    lTime = []
    
    for t in range(0,numOfShots):
        
        # copy SSP waveform for each shot
        iterTime = sspWv[t][0][:].copy()
        
        # last time point for the sequencer
        # this value can be 799.0
        lTimeStamp = sspWv[t][0][-1]
        
        # take the 2nd through 5th to last time point
        # and iterate over them for comparison
        for i,j in enumerate(iterTime.flat[-2:-6:-1]):
            if (j < lTimeStamp) & ((lTimeStamp % j) > 1.0):
                lTime.append(j)
            else:
                continue

            lTimeStamp = j
            
    return lTime

def scaleTime(wave,sspEndTimes,numOfShots):
    ''' Takes:
            waveform of a sequencer (wave),
            vector of end times < 799.0 from the SSP 
            board (sspEndTimes),
            XML file count (numOfShots)
        Returns:
            waveform of a sequencer with end time points
            matching that of the SSP board (modifiedWave),
            count of the number of indices greater than 799.0'''
    
    # Length of input waveform
    waveLen = len(wave[0][0][:])
    
    # Placeholder for index of time values >= 799.0
    idx2Cut = int()
    
    # Count and store the number of time values
    # >= 799.0. This code assumes the count is
    # consistant accross time.
    for t in range(0,numOfShots):
        
        # copy waveform for each shot
        iterTime = wave[t][0][:].copy()
        
        # last time point for the sequencer
        # this value can be 799.0
        lTimeStamp = wave[t][0][-1]
        count = 0
        
        # take the 2nd through 5th to last time point
        # and iterate over them for comparison
        for i,j in enumerate(iterTime.flat[-2:-6:-1]):
            
            if (j < lTimeStamp) & ((lTimeStamp % j) > 1.0):
                
                # replace the first time point most different 
                # from the subsequent one with another.
                wave[t][0][waveLen - 1 - i] = sspEndTimes[t]
                break
            else:
                
                # add to count if the present time point is 
                # not much different from the one before
                count += 1
                continue
            
            lTimeStamp = j
            
        idx2Cut = count
        
    modifiedWave = wave.copy()
    del waveLen, lTimeStamp, iterTime
            
    return modifiedWave, idx2Cut

def waveTruncate(wave,cols2Cut,numOfShots):
    ''' Takes:
            waveform of a sequencer (wave),
            number of time values >= 799.0 (cols2Cut),
            XML file count (numOfShots)
        Returns:
            truncated waveform of a sequencer (truncatedWv)'''
    
    # length of input waveform
    waveLen = len(wave[0][0][:])
    
    # Placeholder with compensation for
    # decreased sizes of the input waveform
    truncatedWv = np.zeros((numOfShots,2,waveLen-cols2Cut))

    for t in range(0,numOfShots):
        iterTime = wave[t][0][:].copy()
        lTimeStamp = wave[t][0][-1]
        truncatedWv[t] = np.delete(wave[t],range(waveLen-cols2Cut,waveLen),axis=1)
            
    return truncatedWv

### Show waveforms on the same time scale

In [65]:
def waveToPlot(wave,t):
    x = wave[t,0,:]
    y = wave[t,1,:]
    
    return x, y

In [66]:
# seqs2Show = [0,1,2,3,4,7]
# l = len(seqs2Show)

# lastSspTimes = sspStopTime(wvm, stopCond)


# def playSequencer(shots):
#     seqPlots = plt.figure(num=1,figsize=[15.0,27.0])
    
#     for i,p in enumerate(seqs2Show):
#         npStore = extractWvfm(wvm, p, stopCond)

#         wv, toCut = scaleTime(npStore,lastSspTimes,stopCond)

#         xtr = waveTruncate(wv,toCut,stopCond)
        
#         xVal, yVal = waveToPlot(xtr,shots)
        
#         subPlt = seqPlots.add_subplot(l,1,i+1)
#         subPlt.plot(xVal,yVal,'b-')
#         subPlt.set_xlabel('Time (us)')
#         subPlt.set_ylabel('Amplitude (a.u.)')
#         subPlt.autoscale(enable=True,axis='x')
#         subPlt.set_title('Sequence {0} Board'.format(seqr[p]))
#         seqPlots.subplots_adjust(hspace=0.5)
        
#         plt.show

### Interactive plots

In [102]:
class StartPage(tk.Frame):
    
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        label = tk.Label(self, text="Start Page", font=LARGE_FONT)
        label.pack(padx=10,pady=10)

        button1 = ttk.Button(self, text="Select directory", 
                            command=self.getFileInfo)
        button2 = ttk.Button(self, text="View Sequencers", 
                            command=lambda: controller.show_frame(PlotterWithZoom))

        button1.pack()
        button2.pack()
        
    def getFileInfo(self):
        curDir = tk.Tk()
        global sourceFolder
        # starting from user's home folder, pick directory in which XMLs exist
        sourceFolder = filedialog.askdirectory(parent=curDir, initialdir=os.getcwd(), 
                                                title="Please select the XML directory")

        curDir.destroy()
    
class PageOne(tk.Frame):
    
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        label = tk.Label(self, text="Plotter", font=LARGE_FONT)
        label.pack(padx=10, pady=10)
        
#         menubar = tk.Menu(self)
#         filemenu = tk.Menu(menubar, tearoff=0)
#         filemenu.add_command(label="New")
#         filemenu.add_command(label="Open")                    
#         menubar.add_cascade(label="Settings", menu=filemenu)
#         controller.config(menu=menubar)
        
        button1 = ttk.Button(self, text="Back to Home", 
                            command=lambda: controller.show_frame(StartPage))
#         button2 = tk.Button(self, text="Page Two", 
#                             command=lambda: controller.show_frame(PageOne))
        
        button1.pack()
#         button2.pack()

In [105]:
""" This snippet of code was copied from 
https://stackoverflow.com/questions/3877774/updating-a-graphs-coordinates-in-matplotlib
 
Additional help for this section should be taken from this YouTube channel
https://www.youtube.com/watch?v=Zw6M-BnAPP0
"""
class PlotterWithZoom(tk.Frame):
    
    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        
        label = tk.Label(self, text="Interactive Plot", font=LARGE_FONT)
        label.pack(padx=10, pady=10)
        
        self.main()
        
         
    def main(self):
        self.seqPlots = Figure(figsize=[15.0,27.0], dpi=100)
         
        self.canvas = FigCanvas(self.seqPlots, self)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack(side='top', fill='both')
        
        toolbar = NavTb2(self.canvas, self)
        toolbar.update()
        toolbar.pack()
        self.canvas._tkcanvas.pack(side='top', fill='both', expand=1)
 
        self.playSequencer(11)
 
        button1 = ttk.Button(self,text='Back to Home',
                                 command=lambda: controller.show_frame(StartPage))
        button1.pack(ipadx=250)
    
    
    def playSequencer(self, shots):        
        xmlList = []
        xmlList.append(sourceFolder)

        # Create placeholder and fill list for xmls
        tempList = []
        xmlFiles = []

        # provide the absolute path of the current directory
        imLoc = xmlList[0] + '/'
        # list all files in the XML directory
        dirList = os.listdir(imLoc)
        # store only XML files
        wont = int()
        if len(fnmatch.filter(dirList,'*.xml*')) > 1:
            filesInDir = fnmatch.filter(dirList,'*.xml.*[^0-9]')
            wont = 0
        elif len(fnmatch.filter(dirList,'*.xml*')) == 1:
            filesInDir = fnmatch.filter(dirList,'*.xml*')
            wont = 1
        elif len(fnmatch.filter(dirList,'*')) > 0:
            filesInDir = []
            wont = 2
            files = fnmatch.filter(dirList,'*')
            for x in files:
                if (ET.parse(x)):
                    filesInDir.append(x)
                else:
                    continue
        else :
            raise UserWarning('Found no XML files or the directory was empty.\n')

        for j,k in enumerate(filesInDir):
            tempList.append(xmlItemize(wont,k))
            xmlFiles.append(k)

        sortedFiles = xmlSort(tempList, xmlFiles)

        # Array for the sorted abs. path for each XML image
        xmlFullPath = []

        # Fill array with XML abs. paths
        x = 0
        stopCond = len(sortedFiles)
        for i,j in sortedFiles:
            xmlFullPath.append(xmlList[0] + '/' + j)


        # Garbage collection of unused variables
        del tempList, xmlList, sortedFiles

        # load an XML for info on first plotter time-point
        wvm = xmlRoot(xmlFullPath,stopCond)

        seqr = {0: 'SSP', 1: 'XGRAD', 2: 'YGRAD', 3: 'ZGRAD',\
                4: 'RHO1', 5: 'RHO2', 6: 'THETA1', 7: 'THETA2'}

        seqs2Show = [0,1,2,3,4,5]
        l = len(seqs2Show)
        lastSspTimes = sspStopTime(wvm, stopCond)

        for i,p in enumerate(seqs2Show):
            npStore = extractWvfm(wvm, p, stopCond)

            wv, toCut = scaleTime(npStore,lastSspTimes,stopCond)

            xtr = waveTruncate(wv,toCut,stopCond)

            xVal, yVal = waveToPlot(xtr,shots)


            self.seqPlots.clear()

            subPlt = self.seqPlots.add_subplot(l,1,i+1)
            subPlt.plot(xVal,yVal,'b-')
            subPlt.set_xlabel('Time (us)')
            subPlt.set_ylabel('Amplitude (a.u.)')
            subPlt.autoscale(enable=True,axis='x')
            subPlt.set_title('Sequence {0} Board'.format(seqr[p]))

            self.seqPlots.subplots_adjust(hspace=0.5)
            self.canvas.draw()
         

In [103]:
class genericGUI(tk.Tk):
    
    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        
        tk.Tk.wm_title(self, "singleSeqInteract")
        
        container = tk.Frame(self)
        container.pack(side="top", fill="both", expand = True)
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)
        
        self.frames = {}
        
        for F in (StartPage, PlotterWithZoom):
            
            frame = F(container, self)

            self.frames[F] = frame

            frame.grid(row=0, column=0, sticky="nsew")

        self.show_frame(StartPage)
        self.centerWindow()

    def show_frame(self, frameToAdd):
        
        frame = self.frames[frameToAdd]
        frame.tkraise()
        
    def centerWindow(self):
        w = 300
        h = 350
        sw = self.winfo_screenwidth()
        sh = self.winfo_screenheight()
        x = (sw - w)/2
        y = (sh - h)/2
        self.geometry('%dx%d+%d+%d' % (w, h, x, y))

### Pick the location of the XML

In [106]:
#if len(sys.argv) != 5:
#    sys.exit('Usage: python t1rProcessing.py <spinLockImLocation> minTSL maxTSL numTSL')

#print 'Script Name: ', sys.argv[0]
#print 'Input File: ', sys.argv[1]
#print 'Minimum TSL: ', sys.argv[2]
#print 'Maximum TSL: ', sys.argv[3]
#print 'TSL Quantity: ', sys.argv[4]

#min_TSL = sys.argv[2]
#max_TSL = sys.argv[3]
#tslPlayed = sys.argv[4]

# Call GUI window for selection of XML directories
# for storage of their full paths
app = genericGUI()
# ani = animation.FuncAnimation(seqPlots,playSequencer, interval=stopCond)
app.mainloop()

In [19]:
sourceFolder

'/Users/nowusu/Desktop/IS10b'